In [ ]:
import requests
import json
import time
import os
import pandas as pd
# Aquí copia y pega tu bearer_token como string
bearer_token = 'AAAAAAAAAAAAAAAAAAAAAATmNAEAAAAAHgDjutXTLigS%2Ffp%2Fr0S9IjMnet0%3DBWAdAMFQJwaJXzumNuL6rGW7imGci7C2sE761Mz9NfAJYmc6rI'
headers = {"Authorization": "Bearer {}".format(bearer_token)}

# Toma como directorio de trabajo la ubicación del archivo (.ipynb) (necesitamos tener ahí mismo las bases de datos)
pwd = os.getcwd()

## Limpieza de Google Forms 

### Objetivo: Importar encuesta y limpiar: 
Archivo resultado:  1_colmex_limpio.csv.    Última descarga 09/04/2020 00:00 hrs 328 registros en total. 310 registros netos.

In [ ]:
# Importamos la base original
original = pd.read_csv(pwd + "/0_google_forms.csv")
# Duplicamos la base para no modificar la original
colmex = original.copy()
# Cambiamos el nombre de las columnas
colmex = colmex.rename(columns={colmex.columns[0]: 'time',
                        colmex.columns[1]: 'status',
                        colmex.columns[2]: 'center',
                        colmex.columns[3]: 'username',
                        colmex.columns[4]: 'id'})
# Seleccionamos las columnas relevantes
colmex = colmex[["id", "username","status","center"]]
# Creamos dos diccionarios para mapear los nombres dic_status y dic_center
dic_status = {'En la comunidad actualmente': 'current',
    'Ex-alumno, ex-becario, ex-profesor, ex-administrativo': 'former'}
dic_center = { 
    "Centro de Estudios Económicos (CEE)": "CEE",
     "Centro de Estudios Históricos (CEH)": "CEH",
     "Centro de Estudios Internacionales (CEI)":"CEI",
     "Centro de Estudios Económicos (CEE)" : "CEE",
     "Centro de Estudios Lingüísticos y Literarios (CELL)" : "CELL",
     "Centro de Estudios de Asia y África (CEAA)" : "CEAA",
     "Centro de Estudios Demográficos, Urbanos y Ambientales (CEDUA)" : "CEDUA",
     "Centro de Estudios Sociológicos (CES)" : "CES",
     "Biblioteca/Administrativo/Idiomas" : "ADM"}
# Remplazamos los valores de acuerdo a los diccionarios
colmex=colmex.replace({"status": dic_status})
colmex=colmex.replace({"center": dic_center})
# Quitamos IDs duplicados
colmex = colmex.drop_duplicates(subset=['id'])
# Quitamos usernames mal escritos con @ al inicio
colmex['username'] = colmex['username'].str.replace('@','')
colmex.reset_index(drop=True)
# Guardamos la base limpia
colmex.to_csv (pwd + "/1_colmex_limpio.csv", index = False)
colmex.info()

## Base limpia y funciones adicionales

### Primer mitad \[0:120\] Diego
### Segunda mitad \[120:237\] Emilio

In [ ]:
# Llamamos la base limpia, tiene 307 observaciones:
colmex_original = pd.read_csv(pwd + "/1_colmex_limpio.csv")
# Respaldamos la base original por si haremos cortes o updtes
colmex = colmex_original
colmex

### Una función que nos da el índice en Colmex al escribir el usuario

In [ ]:
def index_colmex(username):
    try:
        index = colmex_original['username'].loc[lambda x: x==username].index[0]
        print("El index en Colmex de " + username + " es " + str(index))
    except IndexError:
        print ("El usuario no está en Colmex")

### Una función que nos da el usuario y ID en Colmex al escribir el índice

In [ ]:
def username_colmex(index):
    try:
        id = colmex_original['id'].iloc[index]
        user = colmex_original['username'].iloc[index]
        print("El username en Colmex del índice " + str(index) + " es " + user + ", ID: " + str(id))
    except IndexError:
        print ("El índice no está en Colmex")

### Probamos las funciones

In [ ]:
index_colmex("Alextuto")

In [ ]:
username_colmex(286)

## Descarga de datos desde API Twitter

In [ ]:
# Creamos un data frame vacío con las columnas finales
df = pd.DataFrame(columns=['id','username','status','center','target','t_username','t_name'])
# Cargamos el df a actualizar
# df = pd.read_csv(pwd +"/2_red_colmex_completa.csv")
df.info()

In [ ]:
# Hacemos un for loop maravilloso lleno de if's y while's internos y vamos llenado la base df
# Vamos a iterar para todas las observaciones de colmex
for num in range(colmex.shape[0]):
    id =  colmex['id'].iloc[num] # Guardamos el id en una variable
    url = f'https://api.twitter.com/2/users/{id}/following?max_results=1000'  # Generamos el URL del request de 1000 following
    req = requests.get(url,headers=headers) # Hacemos el request
    time.sleep(65) # Esperamos 70 segundos porque podemos hacer 15 request en una ventana de 15 minutos
    if 'data' in req.json().keys(): # Revisamos que sea un request exitoso (cuando son cuentas privadas marca error)
        following = req.json() # Pasamos el json object a un diccionario de Python
        data = dict()
        data['data'] = []
        data['data'].extend(following['data'])  # Pasamos el diccionario a una lista con los datos
        df_loop = pd.json_normalize(data, record_path =['data']) # Convertimos a un pandas dataframe con método normalize
        df_loop = df_loop[['id', 'name', 'username']] # Nos quedamos con las columnas que nos interesan (por si hay extras)
        df_loop = df_loop.rename(columns={"id": "target", # Cambiamos los nombres del target
        "name":"t_name","username":"t_username"})
        df_loop["id"]=colmex['id'].iloc[num] 
        df_loop["username"]=colmex['username'].iloc[num]
        df_loop["status"]=colmex['status'].iloc[num]
        df_loop["center"]=colmex['center'].iloc[num]
        df = df.append(df_loop, ignore_index = True)  # Hacemos append del dataframe que itera (df_loop) al dataframe principal (df)
        while 'next_token' in following['meta']: # Si tiene más de 1,000 followings, revisamos el next_token en los metadatos
            pag_num = following['meta']['next_token'] # Guardamos en next_token en la variable page_num
            url=f'https://api.twitter.com/2/users/{id}/following?max_results=1000&pagination_token={pag_num}' 
            req = requests.get(url,headers=headers) # Hacemos nuevamente el request de la siguiente página
            time.sleep(65) # Volvemos a esperar y se repite el proceso, el while se rompe cuando ya no haya un next_token
            following = req.json()
            data = dict()
            data['data'] = []
            data['data'].extend(following['data'])
            df_loop = pd.json_normalize(data, record_path =['data'])
            df_loop = df_loop.rename(columns={"id": "target",
            "name":"t_name","username":"t_username"})
            df_loop["id"]=colmex['id'].iloc[num]
            df_loop["username"]=colmex['username'].iloc[num]
            df_loop["status"]=colmex['status'].iloc[num]
            df_loop["center"]=colmex['center'].iloc[num]
            df = df.append(df_loop, ignore_index = True)
        else:
            continue
    else:
        continue

## ¿Qué hacer si marca un error? Si no hay errores pasar a sección siguiente

Hasta el momento nos hemos encontrado con dos tipos de error. 
- JSONDecodeError: Expecting value: line 1 column 1 (char 0)
- Gateway Timeout (Too many tries) o algo similar.

Muy probablemente el error se generó con un usuario nuevo y no al recuperar páginas siguientes de un usuario. (Recordando que vamos bajando los datos de 1,000 en 1,000). Por tanto vamos a revisar en cuál nos quedamos y retomar la descarga a partir de ahí.

    1. Revisamos hasta dónde avanzó el código y en qué usuario se quedó.

In [ ]:
df.tail()

    2. Ubicamos el índice del usuario que tuvo el problema 
Probablemente el usuario que tuvo problema fue el siguiente al último guardado en df

In [ ]:
# Índice de último usuario en df
index_colmex(df.iloc[-1,1])

    3. Nos quedamos con el resto de la base partiendo del usuario con problema

Ejemplo: El usuario con índice 30 es el último en df, probablemente el 31 marcó el error, comenzamos de nuevo a partir de él.

In [ ]:
colmex = colmex[280:310]
colmex

    4. Esperamos unos 20 min (por si es saturación de Twitter) y volvemos a correr el for{} con la nueva base

Este nuevo for{} reanuda a partir del usuario con error y añade a df donde se quedó.  

**Importante no remplazar o modificar la base df porque ahí están todos los datos buenos hasta antes del error.**

In [ ]:
for num in range(colmex.shape[0]):
    id =  colmex['id'].iloc[num] # Guardamos el id en una variable
    url = f'https://api.twitter.com/2/users/{id}/following?max_results=1000'  # Generamos el URL del request de 1000 following
    req = requests.get(url,headers=headers) # Hacemos el request
    time.sleep(70) # Esperamos 70 segundos porque podemos hacer 15 request en una ventana de 15 minutos
    if 'data' in req.json().keys(): # Revisamos que sea un request exitoso (cuando son cuentas privadas marca error)
        following = req.json() # Pasamos el json object a un diccionario de Python
        data = dict()
        data['data'] = []
        data['data'].extend(following['data'])  # Pasamos el diccionario a una lista con los datos
        df_loop = pd.json_normalize(data, record_path =['data']) # Convertimos a un pandas dataframe con método normalize
        df_loop = df_loop[['id', 'name', 'username']] # Nos quedamos con las columnas que nos interesan (por si hay extras)
        df_loop = df_loop.rename(columns={"id": "target", # Cambiamos los nombres del target
        "name":"t_name","username":"t_username"})
        df_loop["id"]=colmex['id'].iloc[num] 
        df_loop["username"]=colmex['username'].iloc[num]
        df_loop["status"]=colmex['status'].iloc[num]
        df_loop["center"]=colmex['center'].iloc[num]
        df = df.append(df_loop, ignore_index = True)  # Hacemos append del dataframe que itera (df_loop) al dataframe principal (df)
        while 'next_token' in following['meta']: # Si tiene más de 1,000 followings, revisamos el next_token en los metadatos
            pag_num = following['meta']['next_token'] # Guardamos en next_token en la variable page_num
            url=f'https://api.twitter.com/2/users/{id}/following?max_results=1000&pagination_token={pag_num}' 
            req = requests.get(url,headers=headers) # Hacemos nuevamente el request de la siguiente página
            time.sleep(70) # Volvemos a esperar y se repite el proceso, el while se rompe cuando ya no haya un next_token
            following = req.json()
            data = dict()
            data['data'] = []
            data['data'].extend(following['data'])
            df_loop = pd.json_normalize(data, record_path =['data'])
            df_loop = df_loop.rename(columns={"id": "target",
            "name":"t_name","username":"t_username"})
            df_loop["id"]=colmex['id'].iloc[num]
            df_loop["username"]=colmex['username'].iloc[num]
            df_loop["status"]=colmex['status'].iloc[num]
            df_loop["center"]=colmex['center'].iloc[num]
            df = df.append(df_loop, ignore_index = True)
        else:
            continue
    else:
        continue

Si hay nuevamente un error, repetir el proceso.  
Quizá valdría la pena revisar el perfil del usuario que marca error a ver si tiene algo en especial.  
Si de plano no funciona, nos lo saltamos (subset de nuevo a Colmex a partir del siguiente id)

## Revisamos que llegamos al final de nuestra parte de **colmex** y guardamos la base

In [ ]:
df.tail()

In [ ]:
colmex.tail()

In [ ]:
# El ID debe coincidir con el último usuario en nuestra parte de colmex
index_colmex(df.iloc[-1,1])

In [ ]:
# Guardamos la base como csv 
# Cambiar terminación diego / emilio para separarlas y ubicarlas
df.to_csv (pwd + "/2_red_colmex_completa.csv", index = False)

### Confirmación adicional de que todo está bien, revisar el número de following en nuestra base y en Twitter 

In [ ]:
red_colmex = pd.read_csv(pwd +"/2_red_colmex_completa.csv")

In [ ]:
red_colmex.info()

In [ ]:
# Función que nos dice cuántos following tiene un usuario
def following(username):
    summary = red_colmex.groupby('username').agg(following=('target', 'size')).reset_index().sort_values(['following'], ascending=False)
    return summary.loc[summary['username'] == username]

In [ ]:
# Revisamos cuántos following tiene MGF91
following("MGF91")
# En nuestra base tiene 3539 y viendo en Twitter sigue a 3541 
# (Entre la recolección y ahorita siguió a 2 nuevas personas, ni pedo)

In [ ]:
red_colmex.username.nunique()

In [ ]:
print("Listo :D")

## Base Diego y Emilio

In [ ]:
df.shape
#df[df.username == 'cabrowns']

In [ ]:
df.info()

In [ ]:
#df[df.username == 'cabrowns']

In [ ]:
df_diego=df[0:110556]
df_diego.to_csv (pwd + "/2_red_colmex_diego.csv", index = False)
df_emilio=df[110556:224176]
df_emilio.to_csv (pwd + "/2_red_colmex_emilio.csv", index = False)